In [1]:
import pandas as pd
%matplotlib inline
import glob
import time
import os
import numpy as np
import gc
from contextlib import contextmanager

# sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

from features.features import RideSafetyFeaturesAggregator

import lightgbm as lgb
import xgboost as xgb

In [2]:
DATA_PATH = 'data/safety/safety'

In [3]:
@contextmanager
def timer(task_name="timer"):
    print("----{} started".format(task_name))
    t0 = time.time()
    yield
    print("----{} done in {:.0f} seconds".format(task_name, time.time() - t0))

## Data Fields Description

|      Field      |               Description               |
|:---------------:|:---------------------------------------:|
|    bookingID    |                 trip id                 |
|     Accuracy    |    accuracy inferred by GPS in meters   |
|     Bearing     |          GPS bearing in degree          |
|  acceleration_x |  accelerometer reading at x axis (m/s2) |
|  acceleration_y |  accelerometer reading at y axis (m/s2) |
|  acceleration_z |  accelerometer reading at z axis (m/s2) |
|      gyro_x     |   gyroscope reading in x axis (rad/s)   |
|      gyro_y     |   gyroscope reading in y axis (rad/s)   |
|      gyro_z     |   gyroscope reading in z axis (rad/s)   |
|      second     | time of the record by number of seconds |
|      Speed      |       speed measured by GPS in m/s      |



In [4]:
# load labels
labels_path = '{}/labels'.format(DATA_PATH)
labels = pd.read_csv(glob.glob('{}/*.csv'.format(labels_path))[0])
labels = labels.sort_values(by='bookingID')

In [5]:
print(labels.shape)
labels.head()

(20018, 2)


,bookingID,label
15035,0,0
13312,1,1
996,2,1
2328,4,1
5192,6,0


In [12]:
features_path = '{}/features'.format(DATA_PATH)
features = pd.DataFrame()
for f in glob.glob('{}/*.csv'.format(features_path)):
    print('loading feature: ', f)
    temp = pd.read_csv(f)
    features = pd.concat([features, temp], axis=0)
    break

    
features = features.sort_values(by=['bookingID', 'second'])

loading feature:  data/safety/safety/features\part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv


In [13]:
features.shape

(1613554, 11)

In [14]:
print(features.shape)
features.head()

(1613554, 11)


,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
436147,0,8.0,143.298294,-1.416705,-9.548032,-1.860977,-0.022413,0.005049,-0.025753,3.0,0.228454
314954,0,8.0,143.298294,-0.602780,-9.484927,-1.867706,-0.002221,-0.022451,0.009535,22.0,0.228454
1184808,0,12.0,143.298294,0.761737,-9.937573,-2.080496,-0.023960,-0.108738,0.090643,31.0,0.297795
1327390,0,12.0,144.299423,-1.005032,-9.767700,-1.728189,0.001761,0.013517,-0.017036,36.0,0.297795
1013616,0,12.0,144.299423,-0.909479,-9.798056,-1.886249,0.001773,0.020846,-0.013098,51.0,0.297795


In [231]:
def get_stopping_statistics(df):
    # gets every vehicle stop in a trip and returns its start_time, end_time and diff
    
    # make sure all runs of ones are well-bounded
    bounded = np.hstack(([1], df.Speed.values, [1]))

    log = (bounded < 0.5) * 1
    
    # get 1 at run starts and -1 at run ends
    diffs = np.diff(log)    
  
    # get indices if starts and ends
    run_starts = np.where(diffs > 0)[0]
    run_ends = np.where(diffs < 0)[0]
    
    interval = 7
    end_stops = np.array([run_starts,run_ends,run_ends-run_starts]).T
    end_stops = end_stops.astype(int)[:-1,1]
    end_stops = end_stops[end_stops + interval < len(df.Speed.values) - 1]  
    
    n_stops = len(end_stops)
    
    if n_stops > 1:
        hit = np.zeros(shape=(1 ,n_stops))
        for i in range(n_stops):
            # slope at acceleration    
            start = end_stops[i]
            hit[0, i] =  np.diff([df.Speed.values[start], df.Speed.values[start + interval]])
    else:
        hit = np.array([0])
  
    return [n_stops, hit.mean(), hit.max(), hit.std()]

def get_relative_stopping_time(df):
    last = round(len(df) * 0.05)
    eps = 1

    # determine the stopping ratio of last 5% of a trip
    speed_red = df.Speed.values[len(df.Speed.values) - last:]
    return len(speed_red[speed_red < 0 + eps]) / float(df.second.max()) 

def get_naive_distance(df):
    return (((df['second'].shift(-1) - df['second'])).fillna(0) * df['Speed']).sum()

def get_other_features(df):
    n_stops, hit_mean, hit_max, hit_std = get_stopping_statistics(df)
    naive_dist = get_naive_distance(df)
    stopping_time_ratio = get_relative_stopping_time(df)
    
    d = {
        'n_stops': n_stops,
        'hit_mean': hit_mean,
        'hit_max': hit_max,
        'hit_std': hit_std,
        'naive_distance': naive_dist,
        'stopping_time_ratio': stopping_time_ratio,
    }
    return pd.Series(d, index=['n_stops', 'hit_mean', 'hit_max', 'hit_std', 'naive_distance', 'stopping_time_ratio']) 

In [232]:
def percentile25(x):
    return x.quantile(0.25)

def percentile50(x):
    return x.median()

def percentile75(x):
    return x.quantile(0.75)

aggregate_functions = ['mean', 'min', 'max', 'std', percentile25, percentile50, percentile75]
agg_columns_excluded = ['bookingID', 'second']
agg_dict = {c: aggregate_functions for c in features.columns if c not in agg_columns_excluded}
agg_dict['second'] = ['max']

other_features = features.groupby('bookingID', as_index=True).apply(get_other_features)
other_features = other_features.reset_index()

In [233]:
features_agg = features.groupby(['bookingID'], as_index=True).agg(agg_dict)
features_agg.columns = features_agg.columns.map('_'.join)
features_agg = features_agg.reset_index(drop=False)

features_agg = pd.merge(features_agg, other_features, how='left', on='bookingID')
labels_no_duplicate = labels.drop_duplicates(subset='bookingID')
features_agg = pd.merge(features_agg, labels_no_duplicate, how='left', on='bookingID')

In [17]:
features = features.iloc[:1000]

In [18]:
features.shape

(1000, 11)

In [22]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [23]:
feat = RideSafetyFeaturesAggregator(features)
features_agg = feat.get_aggregated_features()

TypeError: 'DataFrame' object is not callable

In [234]:
feature_columns = [c for c in features_agg.columns.values if c not in ['bookingID', 'label']]
label_column = 'label'

## Modelling Part

In [397]:
class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train, **kwargs):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)

In [435]:
class LGBWrapper(object):
    def __init__(self, params=None):
        self.param = params
        self.num_rounds = params.pop('nrounds', 60000)
        self.early_stop_rounds = params.pop('early_stop_rounds', 2000)

    def train(self, x_train, y_train, **kwargs):
        dtrain = lgb.Dataset(x_train, label=y_train)
        dvalid = lgb.Dataset(kwargs['x_val'], label=kwargs['y_val'])

        watchlist = [dtrain, dvalid]
        self.model = lgb.train(
                  self.param,
                  train_set=dtrain,
                  num_boost_round=self.num_rounds,
                  valid_sets=watchlist,
                  verbose_eval=1000,
                  early_stopping_rounds=self.early_stop_rounds
        )

    def predict(self, x):
        return self.model.predict(x, num_iteration=self.model.best_iteration)

In [439]:
class XGBWrapper(object):
    def __init__(self, params=None):
        self.param = params
        self.nrounds = params.pop('nrounds', 60000)
        self.early_stop_rounds = params.pop('early_stop_rounds', 2000)

    def train(self, x_train, y_train, **kwargs):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        dvalid = xgb.DMatrix(data=kwargs['x_val'], label=kwargs['y_val'])
        
        watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
        
        self.model = xgb.train(dtrain=dtrain, num_boost_round=self.nrounds, evals=watchlist, early_stopping_rounds=self.early_stop_rounds, 
                               verbose_eval=1000, params=self.param)

    def predict(self, x):
        return self.model.predict(xgb.DMatrix(x), ntree_limit=self.model.best_ntree_limit)

In [409]:
NUM_SPLITS = 10
def get_oof(clf, num_splits, X, y, X_test):    
    oof_train = np.zeros((len(X),))
    oof_test = np.zeros((len(X_test),))
    oof_test_skf = np.empty((num_splits, len(X_test)))

    for i, (train_index, test_index) in enumerate(StratifiedKFold(n_splits=num_splits, shuffle=True).split(X, y)):
        print('Training for fold: ', i + 1)
        
        x_tr = X.iloc[train_index, :]
        y_tr = y[train_index]
        x_te = X.iloc[test_index, :]
        y_te = y[test_index]

        clf.train(x_tr, y_tr, x_val=x_te, y_val=y_te)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(X_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [436]:
lgbm_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'nrounds': 50000,
    'early_stop_rounds': 2000,
    # trainable params
    'max_depth': 4,
    'num_leaves': 46,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.9,
    'bagging_freq': 8,
    'learning_rate': 0.019,
    'verbose': 0
}
lgb_wrapper = LGBWrapper(lgbm_params)

with timer('Training LightGBM'):
    lgb_oof_train, lgb_oof_test = get_oof(lgb_wrapper, num_splits=NUM_SPLITS, X=features_agg[feature_columns], y=features_agg[label_column], X_test=features_agg[feature_columns])

----Training LightGBM started
Training for fold:  1
Training until validation scores don't improve for 2000 rounds.
[1000]	training's rmse: 0.355733	valid_1's rmse: 0.391129
[2000]	training's rmse: 0.325592	valid_1's rmse: 0.392536
Early stopping, best iteration is:
[691]	training's rmse: 0.366617	valid_1's rmse: 0.391037
Training for fold:  2
Training until validation scores don't improve for 2000 rounds.
[1000]	training's rmse: 0.356221	valid_1's rmse: 0.391428
[2000]	training's rmse: 0.325814	valid_1's rmse: 0.392936
Early stopping, best iteration is:
[729]	training's rmse: 0.365616	valid_1's rmse: 0.391186
Training for fold:  3
Training until validation scores don't improve for 2000 rounds.
[1000]	training's rmse: 0.356224	valid_1's rmse: 0.39738
[2000]	training's rmse: 0.32534	valid_1's rmse: 0.39972
Early stopping, best iteration is:
[481]	training's rmse: 0.373999	valid_1's rmse: 0.396139
Training for fold:  4
Training until validation scores don't improve for 2000 rounds.
[1000

In [458]:
# XGB
xgb_params = {
    'eval_metric': 'rmse',
    'device': 'cpu',
    'silent': 1,
    'seed': 1337,
    'nrounds': 60000,
    'early_stop_rounds': 2000,
    # trainable params
    'eta': 0.025,
    'subsample': 0.8,
    'colsample_bytree': 0.6000000000000001,
    'gamma': 0.65,
    'max_depth': 5,
    'min_child_weight': 5.0,
    'n_estimators': 500,
}

xgbWrapper = XGBWrapper(xgb_params)

with timer('Training Xgboost'):
    xgb_oof_train, xgb_oof_test = get_oof(xgbWrapper, num_splits=NUM_SPLITS, X=features_agg[feature_columns], y=features_agg[label_column], X_test=features_agg[feature_columns])

----Training Xgboost started
Training for fold:  1
[0]	train-rmse:0.495548	valid-rmse:0.495484
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 2000 rounds.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[1000]	train-rmse:0.310379	valid-rmse:0.392188
[2000]	train-rmse:0.264858	valid-rmse:0.393976
Stopping. Best iteration:
[249]	train-rmse:0.36715	valid-rmse:0.390527

Training for fold:  2
[0]	train-rmse:0.495512	valid-rmse:0.495589
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 2000 rounds.
[1000]	train-rmse:0.311131	valid-rmse:0.396582
[2000]	train-rmse:0.265693	valid-rmse:0.399135
Stopping. Best iteration:
[249]	train-rmse:0.366852	valid-rmse:0.393856

Training for fold:  3
[0]	train-rmse:0.495519	valid-rmse:0.49559
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 2000 rounds.
[1000]	train-rmse:0.30983	valid-rmse:0.396241
[2000]	train-rmse:0.265163	valid-rmse:0.398969
Stopping. Best iteration:
[293]	train-rmse:0.362763	valid-rmse:0.393818

Training for fold:  4
[0]	train-rmse:0.495543	valid-rmse:0.495749
Multiple e

In [608]:
# random forest
rf_params = {
    'n_jobs': -1,
    'n_estimators': 150,
    'max_features': 'auto',
    'min_samples_leaf': 2,
}

random_forest_clf = SklearnWrapper(clf=RandomForestClassifier, seed=1337, params=rf_params)

with timer('Training random forest'):
    rf_oof_train, rf_oof_test = get_oof(random_forest_clf, num_splits=NUM_SPLITS, X=features_agg[feature_columns], y=features_agg[label_column], X_test=features_agg[feature_columns])

----Training random forest started
Training for fold:  1
Training for fold:  2
Training for fold:  3
Training for fold:  4
Training for fold:  5
Training for fold:  6
Training for fold:  7
Training for fold:  8
Training for fold:  9
Training for fold:  10
----Training random forest done in 27 seconds


In [610]:
roc_auc_score(features_agg[label_column].values, rf_oof_train)

0.5802202500546717

In [443]:
# extra trees classifier
et_params = {
    'n_jobs': -1,
    'n_estimators': 150,
    'max_features': 'auto',
    'max_depth': 5,
    'min_samples_leaf': 2,
}
et = SklearnWrapper(clf=ExtraTreesRegressor, seed=1220, params=et_params)

with timer('Training Extra trees'):
    et_oof_train, et_oof_test = get_oof(et, num_splits=NUM_SPLITS, X=features_agg[feature_columns], y=features_agg[label_column], X_test=features_agg[feature_columns])

----Training Extra trees started
Training for fold:  1
Training for fold:  2
Training for fold:  3
Training for fold:  4
Training for fold:  5
Training for fold:  6
Training for fold:  7
Training for fold:  8
Training for fold:  9
Training for fold:  10
----Training Extra trees done in 9 seconds


In [503]:
x_train_ensemble = pd.DataFrame(np.concatenate((et_oof_train, rf_oof_train, lgb_oof_train, xgb_oof_train), axis=1))

In [504]:
# Stack, combine and train ridge regressor
ridge_params = {
    'alpha':50.0, 
    'fit_intercept':True, 
    'normalize':False, 
    'copy_X':True,
    'max_iter':None, 
    'tol':0.001, 
    'solver':'auto', 
    'random_state':1337
}
ridge = SklearnWrapper(clf=Ridge, seed=1337, params=ridge_params)

with timer('Training stacked ridge regression'):
    final_oof_train, final_oof_test = get_oof(ridge, num_splits=NUM_SPLITS, X=x_train_ensemble, y=features_agg[label_column], X_test=x_train_ensemble)

----Training stacked ridge regression started
Training for fold:  1
Training for fold:  2
Training for fold:  3
Training for fold:  4
Training for fold:  5
Training for fold:  6
Training for fold:  7
Training for fold:  8
Training for fold:  9
Training for fold:  10
----Training stacked ridge regression done in 0 seconds


In [505]:
final_oof_train

array([[0.47540958],
       [0.2340191 ],
       [0.35311893],
       ...,
       [0.18955363],
       [0.2266522 ],
       [0.21166314]])

In [601]:
max_auc = 0
max_thres = 0.1
for thres in np.linspace(0.1, 0.99, 89):
    binarized_oof = (final_oof_train >= thres).astype(int)
    if roc_auc_score(features_agg[label_column].values, binarized_oof) > max_auc:
        max_auc = roc_auc_score(features_agg[label_column].values, binarized_oof)
        max_thres = thres
    
print('out of fold AUC score: ', max_auc)
print('max thres: ', max_thres)

out of fold AUC score:  0.6626954940271171
max thres:  0.2415909090909091


In [260]:
importances = []
for col, importance in zip(feature_columns, cf.feature_importances_):
    importances.append((col, importance))
importances = sorted(importances, key=lambda tup: tup[1], reverse=True)

In [261]:
print('Feature importances: ')
for col, imp in importances:
    print('{}: {}'.format(col, imp))

Feature importances: 
second_max: 0.586876679698496
acceleration_z_std: 0.06800277905166972
naive_distance: 0.04634874356006946
Bearing_std: 0.03098892519411976
Speed_mean: 0.027120287051619035
Speed_percentile50: 0.026338847320603584
Speed_max: 0.01993031517813635
acceleration_x_std: 0.017092960057855374
Speed_percentile75: 0.014449576034211799
acceleration_y_std: 0.014259053758871394
Speed_percentile25: 0.012849472096093991
hit_max: 0.012547453519226698
gyro_z_percentile75: 0.009458499941123773
gyro_z_std: 0.006138746285482801
gyro_z_percentile25: 0.005762337790266419
gyro_x_std: 0.005500720415370757
acceleration_y_percentile25: 0.005054981596664897
Accuracy_max: 0.00413861376900486
acceleration_x_max: 0.004132401312090438
gyro_y_max: 0.003509123216699895
hit_std: 0.0034302448263387767
Speed_std: 0.003406858910701936
Accuracy_percentile50: 0.0031097575090448886
acceleration_y_percentile50: 0.003014979187306925
gyro_z_max: 0.0029292181611606026
Bearing_percentile25: 0.0028581906967139